In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle, os, sys
import matplotlib as mpl
from scipy.interpolate import SmoothBivariateSpline
from matplotlib.colors import LogNorm, Normalize

In [2]:
plt.rcParams.update({'font.size': 20})
mpl.rcParams['mathtext.fontset'] = 'custom'
mpl.rcParams['mathtext.rm'] = 'Bitstream Vera Sans'
mpl.rcParams['mathtext.it'] = 'Bitstream Vera Sans:italic'
mpl.rcParams['mathtext.bf'] = 'Bitstream Vera Sans:bold'

mpl.rcParams['patch.linewidth'] = 2.0
mpl.rcParams['lines.linewidth'] = 2.0

In [3]:
from get_weight_total_and_diff import *

In [4]:
indir = '/home/mliubar/projects/rpp-kenclark/hignight/atmo_flux'
numu = pickle.load(open(os.path.join(indir, 'Level5p_IC86.2013_genie_numu.14600.rlogL.pckl'),'rb'),
                   encoding='latin1')
nue = pickle.load(open(os.path.join(indir, 'Level5p_IC86.2013_genie_nue.12600.rlogL.pckl'),'rb'),
                  encoding='latin1')

# xsec type separation

In [5]:
nu = {}
for k in numu.keys():
    if k == 'weight_e': continue
    if k == 'weight_mu': 
        nu['weight_emu'] = np.concatenate((nue['weight_e'],numu['weight_mu']))
        continue
    nu[k] = np.concatenate((nue[k],numu[k]))

In [6]:
def myargwh(cond):
    return np.transpose(np.argwhere(cond))[0]

In [7]:
ind0 = myargwh(np.abs(nu['ma_res']).sum(axis=1) == 0)
ind1 = myargwh(np.abs(nu['ma_qe'][ind0]).sum(axis=1) == 0)
dis = ind0[ind1]

In [8]:
CCdis = dis[myargwh(nu['interaction'][dis]==1.)]
NCdis = dis[myargwh(nu['interaction'][dis]==2.)]
CCnotdis = ~dis[myargwh(nu['interaction'][~dis]==1.)]
NCnotdis = ~dis[myargwh(nu['interaction'][~dis]==2.)]

In [9]:
CCdisNu = CCdis[myargwh(nu['ptype'][CCdis]>0)]
NCdisNu = NCdis[myargwh(nu['ptype'][NCdis]>0)]
CCnotdisNu = CCnotdis[myargwh(nu['ptype'][CCnotdis]>0)]
NCnotdisNu = NCnotdis[myargwh(nu['ptype'][NCnotdis]>0)]

CCdisNuBar = CCdis[myargwh(nu['ptype'][CCdis]<0)]
NCdisNuBar = NCdis[myargwh(nu['ptype'][NCdis]<0)]
CCnotdisNuBar = CCnotdis[myargwh(nu['ptype'][CCnotdis]<0)]
NCnotdisNuBar = NCnotdis[myargwh(nu['ptype'][NCnotdis]<0)]

In [10]:
val = np.log10(nu['energy'])
glob_w = nu['weight_emu']

In [11]:
# total + differential
def get_w(weight=glob_w, nyears=1, 
          corr={'CCnu':False, 'CCnubar':False, 'NCnu':False, 'NCnubar':False}):
    
    if nyears == 0:
        m = 1.
    else:
        m = m = 365*24*60.*60.*nyears
        
    total_xsec_extr = 'constant'
    total_xsec_lgE_min = 2.
        
    wf = np.ones(len(glob_w))
    
    if corr['CCnu']:
        wf[CCdisNu] = wf[CCdisNu]\
        *get_weight_diff(nu['energy'][CCdisNu], 
                         nu['GENIE_ys'][CCdisNu], 
                         xs_type='NuCC', 
                         coef=1.)\
        *get_weight_total(nu['energy'][CCdisNu], 
                          nu='Nu', current='CC', 
                          ext_type=total_xsec_extr,
                          lgE_min_cust=total_xsec_lgE_min)
    if corr['CCnubar']:
        wf[CCdisNuBar] = wf[CCdisNuBar]\
        *get_weight_diff(nu['energy'][CCdisNuBar], 
                         nu['GENIE_ys'][CCdisNuBar], 
                         xs_type='NuBarCC', 
                         coef=1.)\
        *get_weight_total(nu['energy'][CCdisNuBar], 
                          nu='NuBar', current='CC', 
                          ext_type=total_xsec_extr,
                          lgE_min_cust=total_xsec_lgE_min)
    if corr['NCnu']:
        wf[NCdisNu] = wf[NCdisNu]\
        *get_weight_diff(nu['energy'][NCdisNu], 
                         nu['GENIE_ys'][NCdisNu], 
                         xs_type='NuNC', 
                         coef=1.)\
        *get_weight_total(nu['energy'][NCdisNu], 
                          nu='Nu', current='NC', 
                          ext_type=total_xsec_extr,
                          lgE_min_cust=total_xsec_lgE_min)
    if corr['NCnubar']:
        wf[NCdisNuBar] = wf[NCdisNuBar]\
        *get_weight_diff(nu['energy'][NCdisNuBar], 
                         nu['GENIE_ys'][NCdisNuBar], 
                         xs_type='NuBarNC', 
                         coef=1.)\
       *get_weight_total(nu['energy'][NCdisNuBar], 
                         nu='NuBar', current='NC', 
                         ext_type=total_xsec_extr,
                         lgE_min_cust=total_xsec_lgE_min)

    wr = weight*m*wf
    return wr, m

# plots

In [17]:
lgE_r = [0., 3.]
nb = 20

In [14]:
asp_r = 1./12.*6.

left, width = 0.33, 0.8
bottom, height = 0.12, 0.7
bottom_h = bottom + height + 0.02
left_h = left + width + 0.02

rect1 = [left*asp_r, bottom+0.3, width, height-0.2]
rect2 = [left*asp_r, bottom, width, 0.28]

In [20]:
w_init, m = get_w(nyears=1.)
w_corr, _ = get_w(nyears=1., 
                  corr={'CCnu':True, 'CCnubar':True, 'NCnu':True, 'NCnubar':True})

In [24]:
np.save('dragon_y_for_plots.npy',[{'lgE':np.log10(nu['energy']),'y':nu['GENIE_ys'],
                                  'weights':w_init,'weights_corr':w_corr,
                                  'CCdisNu':CCdisNu, 'CCdisNuBar':CCdisNuBar,
                                  'NCdisNu':NCdisNu, 'NCdisNuBar':NCdisNuBar}])